In [1]:
import os
import uuid

import numpy as np

from pprint import pprint
from datasets import load_dataset
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Building embeddings

In [2]:
data = list(load_dataset("stanfordnlp/imdb", split="train")['text'])
pprint(data[20])

('If the crew behind "Zombie Chronicles" ever read this, here\'s some advice '
 'guys: <br /><br />1. In a "Twist Ending"-type movie, it\'s not a good idea '
 'to insert close-ups of EVERY DEATH IN THE MOVIE in the opening credits. That '
 "tends to spoil the twists, y'know...? <br /><br />2. I know you produced "
 'this on a shoestring and - to be fair - you worked miracles with your budget '
 'but please, hire people who can actually act. Or at least, walk, talk and '
 "gesture at the same time. Joe Haggerty, I'm looking at you...<br /><br />3. "
 "If you're going to set a part of your movie in the past, only do this if you "
 'have the props and costumes of the time.<br /><br />4. Twist endings are '
 "supposed to be a surprise. Sure, we don't want twists that make no sense, "
 'but signposting the "reveal" as soon as you introduce a character? That\'s '
 'not a great idea.<br /><br />Kudos to the guys for trying, but in all '
 "honesty, I'd rather they hadn't...<br /><br />Only for

In [3]:
embedding_model = SentenceTransformer(
    "paraphrase-MiniLM-L3-v2",
    model_kwargs={'dtype': 'float16'}
)

In [4]:
if os.path.exists("imdb_example_files"):
    embeddings = np.load("imdb_example_files/embeddings.npy")
else:
    embeddings = embedding_model.encode(data, normalize_embeddings=True)
    os.mkdir("imdb_example_files")
    np.save("imdb_example_files/embeddings", embeddings)
    with open("imdb_example_files/.gitignore", "w") as f:
        f.write("embeddings.npy\n")

## Vector database

In [ ]:
client = QdrantClient(":memory:")
embedding_size = embeddings.shape[1]

client.create_collection(
    collection_name="imdb",
    on_disk_payload=True,
    vectors_config=models.VectorParams(
        size=embedding_size,
        distance=models.Distance.COSINE,
        on_disk=True
    )
)

True

In [8]:
points = [
    models.PointStruct(
        id=str(uuid.uuid4()),
        vector=list(embeddings[i]),
        payload={"text": data[i]}
    )
    for i in range(len(embeddings))
]
client.upsert(collection_name="imdb", points=points)

/tmp/ipykernel_179178/1207423332.py:9: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 25000 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(collection_name="imdb", points=points)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)